In [37]:
from dataset import get_dataset
from torchvision.datasets import Caltech101
import matplotlib.pyplot as plt

from PIL import Image

In [47]:
dataset = Caltech101("data", download=False)
class_map = dataset.categories

In [48]:
dataset[1000][0], class_map[dataset[1000][1]]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=128x192>, 'Leopards')

In [19]:
train_dataset.categories

AttributeError: 'Subset' object has no attribute 'categories'